# Pandas Data Cleaning and Manipulation Tutorial

This notebook demonstrates essential `pandas` methods for data cleaning and manipulation. You are likely to use some of these methods in your assignments and projects.

**Learning Objectives:**
- Load and inspect data with pandas
- Handle missing values
- Filter and select data
- Transform and create new columns
- Group and aggregate data
- Merge and combine datasets
- Clean and standardize data

## 1. Setup and Import Libraries

In [8]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime

# Set display options for better readability
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 50)        # Show up to 50 rows
pd.set_option('display.width', None)        # Auto-detect display width
pd.set_option('display.max_colwidth', 50)   # Limit column width

## 2. Creating Sample Data

We'll create a sample dataset with various data quality issues to demonstrate cleaning techniques. **You don't need to change the code below.**

In [9]:
# Create sample data with intentional issues for demonstration
np.random.seed(42)  # For reproducibility

data = {
    'student_id': range(1, 21),
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve', 'Frank', 'Grace', 'Henry', 
             'Ivy', 'Jack', 'Kate', 'Liam', 'Mia', 'Noah', 'Olivia', 'Paul', 
             'Quinn', 'Rachel', 'Sam', 'Tina'],
    'age': [20, 21, np.nan, 19, 22, 20, np.nan, 23, 21, 20, 19, 22, 20, 21, np.nan, 20, 21, 19, 22, 20],
    'grade': ['A', 'B', 'A', 'C', 'B', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'A', 'C', 'B', 'A', 'B', 'C', 'A'],
    'score': [95, 85, 92, 72, 88, 90, 87, 75, 94, 82, 70, 93, 86, 91, 68, 84, 96, 83, 73, 89],
    'department': ['CS', 'Math', 'CS', 'Physics', 'Math', 'CS', 'Math', 'Physics', 
                   'CS', 'Math', 'Physics', 'CS', 'Math', 'CS', 'Physics', 'Math', 
                   'CS', 'Math', 'Physics', 'CS'],
    'enrollment_date': ['2023-01-15', '2023-01-20', '2023-02-01', '2023-01-10', '2023-02-15',
                        '2023-01-05', '2023-02-10', '2023-01-25', '2023-02-05', '2023-01-30',
                        '2023-02-20', '2023-01-12', '2023-02-08', '2023-01-18', '2023-02-12',
                        '2023-01-22', '2023-02-03', '2023-01-28', '2023-02-18', '2023-01-08'],
    'email': ['alice@email.com', 'bob@email.com', 'charlie@email.com', 'diana@email.com',
              'eve@email.com', 'frank@email.com', 'grace@email.com', 'henry@email.com',
              'ivy@email.com', 'jack@email.com', 'kate@email.com', 'liam@email.com',
              'mia@email.com', 'noah@email.com', 'olivia@email.com', 'paul@email.com',
              'quinn@email.com', 'rachel@email.com', 'sam@email.com', 'tina@email.com']
}

# Create DataFrame
df = pd.DataFrame(data)

# Introduce some data quality issues
df.loc[2, 'name'] = '  Charlie  '  # Extra whitespace
df.loc[5, 'grade'] = 'a'  # Lowercase inconsistency
df.loc[8, 'department'] = 'cs'  # Lowercase inconsistency
df.loc[12, 'score'] = 150  # Outlier (impossible score)
df.loc[15, 'email'] = 'invalid-email'  # Invalid email format

print("Sample dataset created!")
print(f"\nDataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

Sample dataset created!

Dataset shape: (20, 8)

First few rows:


,student_id,name,age,grade,score,department,enrollment_date,email
0,1,Alice,20.0,A,95,CS,2023-01-15,alice@email.com
1,2,Bob,21.0,B,85,Math,2023-01-20,bob@email.com
2,3,Charlie,NaN,A,92,CS,2023-02-01,charlie@email.com
3,4,Diana,19.0,C,72,Physics,2023-01-10,diana@email.com
4,5,Eve,22.0,B,88,Math,2023-02-15,eve@email.com


## 3. Data Inspection Methods

Before cleaning, it's crucial to understand your data. Here are essential inspection methods:

### 3.1 `df.info()` - Get DataFrame Information

**What it does:** Provides a concise summary of the DataFrame including:
- Number of non-null values in each column
- Data types of each column
- Memory usage

**When to use:** First step after loading data to understand structure and identify missing values.

In [10]:
# Display DataFrame information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   student_id       20 non-null     int64  
 1   name             20 non-null     object 
 2   age              17 non-null     float64
 3   grade            20 non-null     object 
 4   score            20 non-null     int64  
 5   department       20 non-null     object 
 6   enrollment_date  20 non-null     object 
 7   email            20 non-null     object 
dtypes: float64(1), int64(2), object(5)
memory usage: 1.4+ KB


### 3.2 `df.describe()` - Statistical Summary

**What it does:** Generates descriptive statistics for numeric columns:
- Count, mean, standard deviation
- Min, max, quartiles (25%, 50%, 75%)

**When to use:** To understand the distribution of numeric variables and identify potential outliers.

In [11]:
# Statistical summary for numeric columns
df.describe()

,student_id,age,score
count,20.00000,17.000000,20.000000
mean,10.50000,20.588235,87.850000
std,5.91608,1.175735,17.027146
min,1.00000,19.000000,68.000000
25%,5.75000,20.000000,80.250000
50%,10.50000,20.000000,87.500000
75%,15.25000,21.000000,92.250000
max,20.00000,23.000000,150.000000


### 3.3 `df.head()` and `df.tail()` - View First/Last Rows

**What it does:** 
- `head(n)`: Returns first n rows (default: 5)
- `tail(n)`: Returns last n rows (default: 5)

**When to use:** Quick visual inspection of data structure and values.

In [12]:
# View first 5 rows
print("First 5 rows:")
print(df.head())

print("\n" + "="*50 + "\n")

# View last 5 rows
print("Last 5 rows:")
print(df.tail())

First 5 rows:
   student_id         name   age grade  score department enrollment_date  \
0           1        Alice  20.0     A     95         CS      2023-01-15   
1           2          Bob  21.0     B     85       Math      2023-01-20   
2           3    Charlie     NaN     A     92         CS      2023-02-01   
3           4        Diana  19.0     C     72    Physics      2023-01-10   
4           5          Eve  22.0     B     88       Math      2023-02-15   

               email  
0    alice@email.com  
1      bob@email.com  
2  charlie@email.com  
3    diana@email.com  
4      eve@email.com  


Last 5 rows:
    student_id    name   age grade  score department enrollment_date  \
15          16    Paul  20.0     B     84       Math      2023-01-22   
16          17   Quinn  21.0     A     96         CS      2023-02-03   
17          18  Rachel  19.0     B     83       Math      2023-01-28   
18          19     Sam  22.0     C     73    Physics      2023-02-18   
19          20  

### 3.4 `df.isna()` - Check for Missing Values

**What it does:** Returns a DataFrame of boolean values indicating missing data (True = missing, False = present).

**Note:** `isna()` and `isnull()` are identical - use either one depending on your `pandas` version.

**When to use:** To identify which cells contain missing values before deciding how to handle them.

In [13]:
# Check for missing values (returns True/False for each cell)
print("Missing values check:")
print(df.isna().head())

print("\n" + "="*50 + "\n")

# Count missing values per column
print("Missing values count per column:")
print(df.isna().sum())

Missing values check:
   student_id   name    age  grade  score  department  enrollment_date  email
0       False  False  False  False  False       False            False  False
1       False  False  False  False  False       False            False  False
2       False  False   True  False  False       False            False  False
3       False  False  False  False  False       False            False  False
4       False  False  False  False  False       False            False  False


Missing values count per column:
student_id         0
name               0
age                3
grade              0
score              0
department         0
enrollment_date    0
email              0
dtype: int64


### 3.5 `df.value_counts()` - Count Unique Values for a Column

**What it does:** Returns counts of unique values in a Series (column). It is very common when you are dealing with categorical variables. You can use it to understand the distribution of categorical variables or identify unexpected values.

In [14]:
# Count unique values in 'grade' column
print("Value counts for 'grade' column:")
print(df['grade'].value_counts())


Value counts for 'grade' column:
grade
A    7
B    7
C    5
a    1
Name: count, dtype: int64


## 4. Data Selection and Filtering

Selecting and filtering data is fundamental to data analysis:

### 4.1 Column Selection

**Methods:**
- `df['column_name']` - Select single column (returns Series)
- `df[['col1', 'col2']]` - Select multiple columns (returns DataFrame)
- `df.loc[]` - Label-based selection
- `df.iloc[]` - Integer position-based selection

**When to use:** To focus on specific variables or create subsets of your data.

In [15]:
# Example 1: Select single column (returns Series)
print("Single column (Series):")
print(type(df['name']))
print(df['name'].head())

print("\n" + "="*50 + "\n")

# Example 2: Select multiple columns (returns DataFrame)
print("Multiple columns (DataFrame):")
print(type(df[['name', 'age', 'score']]))
print(df[['name', 'age', 'score']].head())

print("\n" + "="*50 + "\n")

# Example 3: Using loc for label-based selection
print("Using loc to select rows and columns:")
print(df.loc[0:4, ['name', 'score']])  # Rows 0-4, specific columns

Single column (Series):
<class 'pandas.core.series.Series'>
0          Alice
1            Bob
2      Charlie  
3          Diana
4            Eve
Name: name, dtype: object


Multiple columns (DataFrame):
<class 'pandas.core.frame.DataFrame'>
          name   age  score
0        Alice  20.0     95
1          Bob  21.0     85
2    Charlie     NaN     92
3        Diana  19.0     72
4          Eve  22.0     88


Using loc to select rows and columns:
          name  score
0        Alice     95
1          Bob     85
2    Charlie       92
3        Diana     72
4          Eve     88


### 4.2 Boolean Indexing / Filtering

**What it does:** Filter rows based on conditions.

**Syntax:** `df[condition]` or `df.loc[condition]`

**When to use:** To subset data based on criteria (e.g., filter by date range, value thresholds, categories).

In [16]:
# Example 1: Filter rows where score > 90
high_scores = df[df['score'] > 90]
print("Students with score > 90:")
print(high_scores[['name', 'score', 'grade']])

print("\n" + "="*50 + "\n")

# Example 2: Multiple conditions using & (and) or | (or)
# Note: Each condition must be in parentheses
cs_students_high_score = df[(df['department'] == 'CS') & (df['score'] > 85)]
print("CS students with score > 85:")
print(cs_students_high_score[['name', 'department', 'score']])

print("\n" + "="*50 + "\n")

# Example 3: Using isin() for multiple values
math_or_physics = df[df['department'].isin(['Math', 'Physics'])]
print("Students in Math or Physics:")
print(math_or_physics[['name', 'department']].head())

Students with score > 90:
           name  score grade
0         Alice     95     A
2     Charlie       92     A
8           Ivy     94     A
11         Liam     93     A
12          Mia    150     B
13         Noah     91     A
16        Quinn     96     A


CS students with score > 85:
           name department  score
0         Alice         CS     95
2     Charlie           CS     92
5         Frank         CS     90
11         Liam         CS     93
13         Noah         CS     91
16        Quinn         CS     96
19         Tina         CS     89


Students in Math or Physics:
    name department
1    Bob       Math
3  Diana    Physics
4    Eve       Math
6  Grace       Math
7  Henry    Physics


## 5. Data Transformation

Transform data to create new columns or modify existing ones:

### 6.1 Creating New Columns

**What it does:** Add new columns based on calculations or transformations of existing columns.

**Methods:**
- `df['new_col'] = values` - Direct assignment
- `df.assign()` - Method chaining friendly
- `df.apply()` - Apply function to rows/columns

**When to use:** To create derived variables, perform calculations, or add metadata.

In [17]:
# Example 1: Create new column with direct assignment
df_copy = df.copy()
df_copy['score_percentage'] = df_copy['score']  # Already a percentage, but renaming for clarity
df_copy['pass_fail'] = df_copy['score'].apply(lambda x: 'Pass' if x >= 70 else 'Fail')
print("New columns created:")
print(df_copy[['name', 'score', 'score_percentage', 'pass_fail']].head())

print("\n" + "="*50 + "\n")

# Example 2: Using assign() for method chaining
df_with_category = df.assign(
    performance_category=lambda x: pd.cut(x['score'], 
                                         bins=[0, 70, 85, 100], 
                                         labels=['Low', 'Medium', 'High'])
)
print("Using assign() to create performance category:")
print(df_with_category[['name', 'score', 'performance_category']].head())

New columns created:
          name  score  score_percentage pass_fail
0        Alice     95                95      Pass
1          Bob     85                85      Pass
2    Charlie       92                92      Pass
3        Diana     72                72      Pass
4          Eve     88                88      Pass


Using assign() to create performance category:
          name  score performance_category
0        Alice     95                 High
1          Bob     85               Medium
2    Charlie       92                 High
3        Diana     72               Medium
4          Eve     88                 High


### 6.2 `df.apply()` - Apply Function to Rows or Columns

**What it does:** Apply a function along an axis (rows or columns) of the DataFrame.

**Key parameters:**
- `func`: Function to apply
- `axis`: 0 (apply to each column) or 1 (apply to each row)
- `args`: Additional positional arguments to pass to function

**When to use:** When you need to apply a custom function that can't be vectorized, or when working with complex row/column operations. You can self define a data cleaning function and apply it to the dataframe.

In [18]:
# Example 1: Apply function to each row
# Here we are converting to the numeric grade to a letter grade. We can define a function with the conversion rules and apply it to the dataframe.
def calculate_grade_letter(score):
    """Convert numeric score to letter grade"""
    if score >= 90:
        return 'A'
    elif score >= 80:
        return 'B'
    elif score >= 70:
        return 'C'
    else:
        return 'D'

df_copy2 = df.copy()
df_copy2['calculated_grade'] = df_copy2['score'].apply(calculate_grade_letter)
print("Applying function to create calculated grade:")
print(df_copy2[['name', 'score', 'grade', 'calculated_grade']].head())

print("\n" + "="*50 + "\n")

# Example 2: Apply function to each column (axis=0)
print("Mean of each numeric column:")
print(df.select_dtypes(include=[np.number]).apply(np.mean, axis=0))

Applying function to create calculated grade:
          name  score grade calculated_grade
0        Alice     95     A                A
1          Bob     85     B                B
2    Charlie       92     A                A
3        Diana     72     C                C
4          Eve     88     B                B


Mean of each numeric column:
student_id    10.500000
age           20.588235
score         87.850000
dtype: float64


### 6.3 `df.map()` - Map Values Using Dictionary or Function

**What it does:** Map values of a Series according to an input mapping (dict, Series, or function).

**When to use:** To replace values based on a mapping (e.g., rename categories, encode values).

In [19]:
# Example: Map department codes to full names
dept_mapping = {
    'CS': 'Computer Science',
    'Math': 'Mathematics',
    'Physics': 'Physics'
}

df_copy3 = df.copy()
df_copy3['department_full'] = df_copy3['department'].map(dept_mapping)
print("Mapping department codes to full names:")
print(df_copy3[['name', 'department', 'department_full']].head())

Mapping department codes to full names:
          name department   department_full
0        Alice         CS  Computer Science
1          Bob       Math       Mathematics
2    Charlie           CS  Computer Science
3        Diana    Physics           Physics
4          Eve       Math       Mathematics


### 6.4 `df.replace()` - Replace Values

**What it does:** Replace values in DataFrame or Series.

**When to use:** To correct data entry errors, standardize values, or replace specific values.

In [20]:
# Example: Replace values (fixing inconsistencies)
df_copy4 = df.copy()
print("Before replacement:")
print(df_copy4['grade'].value_counts())

# Replace lowercase 'a' with uppercase 'A'
df_copy4['grade'] = df_copy4['grade'].replace('a', 'A')
print("\nAfter replacing 'a' with 'A':")
print(df_copy4['grade'].value_counts())

Before replacement:
grade
A    7
B    7
C    5
a    1
Name: count, dtype: int64

After replacing 'a' with 'A':
grade
A    8
B    7
C    5
Name: count, dtype: int64


## 7. String Operations

Pandas provides vectorized string operations through the `.str` accessor:

### 7.1 `df.str.strip()` - Remove Whitespace

**What it does:** Remove leading and trailing whitespace from string values.

**When to use:** To clean up data entry inconsistencies (extra spaces).

In [21]:
# Example: Strip whitespace from names
df_copy5 = df.copy()
print("Before stripping (showing name with extra spaces):")
print(f"Name at index 2: '{df_copy5.loc[2, 'name']}'")

df_copy5['name'] = df_copy5['name'].str.strip()
print(f"\nAfter stripping: '{df_copy5.loc[2, 'name']}'")

Before stripping (showing name with extra spaces):
Name at index 2: '  Charlie  '

After stripping: 'Charlie'


### 7.2 `df.str.upper()` / `df.str.lower()` - Case Conversion

**What it does:** Convert strings to uppercase or lowercase.

**When to use:** To standardize case for categorical variables or text data.

In [22]:
# Example: Standardize department names to uppercase
df_copy6 = df.copy()
print("Before standardization:")
print(df_copy6['department'].value_counts())

df_copy6['department'] = df_copy6['department'].str.upper()
print("\nAfter converting to uppercase:")
print(df_copy6['department'].value_counts())

Before standardization:
department
CS         7
Math       7
Physics    5
cs         1
Name: count, dtype: int64

After converting to uppercase:
department
CS         8
MATH       7
PHYSICS    5
Name: count, dtype: int64


### 7.3 `df.str.contains()` - Check for Substring

**What it does:** Check if each string contains a pattern (returns boolean Series).

**When to use:** To filter or flag rows based on text patterns (e.g., find emails from specific domain).

In [23]:
# Example: Find rows where email contains '@email.com'
valid_emails = df[df['email'].str.contains('@email.com', na=False)]
print("Rows with valid email format:")
print(valid_emails[['name', 'email']].head())

print("\n" + "="*50 + "\n")

# Find invalid emails
invalid_emails = df[~df['email'].str.contains('@email.com', na=False)]
print("Rows with invalid email format:")
print(invalid_emails[['name', 'email']])

Rows with valid email format:
          name              email
0        Alice    alice@email.com
1          Bob      bob@email.com
2    Charlie    charlie@email.com
3        Diana    diana@email.com
4          Eve      eve@email.com


Rows with invalid email format:
    name          email
15  Paul  invalid-email


## 8. Data Type Conversion

Converting data types is crucial for proper analysis:

### 8.1 `pd.to_datetime()` - Convert to DateTime

**What it does:** Convert string or numeric values to datetime objects.

**When to use:** When working with dates/times stored as strings, or when dates need to be parsed from various formats.

In [24]:
# Example: Convert enrollment_date from string to datetime
df_copy7 = df.copy()
print("Before conversion:")
print(f"Type: {df_copy7['enrollment_date'].dtype}")
print(df_copy7[['name', 'enrollment_date']].head())

df_copy7['enrollment_date'] = pd.to_datetime(df_copy7['enrollment_date'])
print("\nAfter conversion:")
print(f"Type: {df_copy7['enrollment_date'].dtype}")
print(df_copy7[['name', 'enrollment_date']].head())

print("\n" + "="*50 + "\n")

# Now we can extract date components
df_copy7['enrollment_month'] = df_copy7['enrollment_date'].dt.month
df_copy7['enrollment_year'] = df_copy7['enrollment_date'].dt.year
print("Extracted date components:")
print(df_copy7[['name', 'enrollment_date', 'enrollment_month', 'enrollment_year']].head())

Before conversion:
Type: object
          name enrollment_date
0        Alice      2023-01-15
1          Bob      2023-01-20
2    Charlie        2023-02-01
3        Diana      2023-01-10
4          Eve      2023-02-15

After conversion:
Type: datetime64[ns]
          name enrollment_date
0        Alice      2023-01-15
1          Bob      2023-01-20
2    Charlie        2023-02-01
3        Diana      2023-01-10
4          Eve      2023-02-15


Extracted date components:
          name enrollment_date  enrollment_month  enrollment_year
0        Alice      2023-01-15                 1             2023
1          Bob      2023-01-20                 1             2023
2    Charlie        2023-02-01                 2             2023
3        Diana      2023-01-10                 1             2023
4          Eve      2023-02-15                 2             2023


### 8.2 `df.astype()` - Convert Data Types

**What it does:** Cast a pandas object to a specified dtype.

**When to use:** To convert between numeric types, convert strings to numbers, or change data types for memory efficiency.

In [25]:
# Example: Convert score from int64 to float64
df_copy8 = df.copy()
print("Before conversion:")
print(f"Score dtype: {df_copy8['score'].dtype}")

df_copy8['score'] = df_copy8['score'].astype('float64')
print(f"\nAfter conversion:")
print(f"Score dtype: {df_copy8['score'].dtype}")

Before conversion:
Score dtype: int64

After conversion:
Score dtype: float64


## 9. Grouping and Aggregation

Grouping allows you to split data into groups and apply functions to each group. This is a very common and useful operation in the exploration analysis part.

### 9.1 `df.groupby()` - Group Data

**What it does:** Split data into groups based on some criteria, then apply a function to each group independently.

**When to use:** To calculate statistics by category, aggregate data, or perform group-level transformations.

In [26]:
# Example 1: Group by department and calculate mean score
dept_stats = df.groupby('department')['score'].mean()
print("Mean score by department:")
print(dept_stats)

print("\n" + "="*50 + "\n")

# Example 2: Multiple aggregations
dept_summary = df.groupby('department').agg({
    'score': ['mean', 'std', 'min', 'max', 'count'],
    'age': 'mean'
})
print("Summary statistics by department:")
print(dept_summary)

print("\n" + "="*50 + "\n")

# Example 3: Group by multiple columns
grade_dept_stats = df.groupby(['department', 'grade'])['score'].mean()
print("Mean score by department and grade:")
print(grade_dept_stats)

Mean score by department:
department
CS         92.285714
Math       94.142857
Physics    71.600000
cs         94.000000
Name: score, dtype: float64


Summary statistics by department:
                score                                  age
                 mean        std min  max count       mean
department                                                
CS          92.285714   2.563480  89   96     7  20.666667
Math        94.142857  24.721304  82  150     7  20.333333
Physics     71.600000   2.701851  68   75     5  20.750000
cs          94.000000        NaN  94   94     1  21.000000


Mean score by department and grade:
department  grade
CS          A        92.666667
            a        90.000000
Math        B        94.142857
Physics     C        71.600000
cs          A        94.000000
Name: score, dtype: float64


## 10. Sorting Data

### 10.1 `df.sort_values()` - Sort by Column Values

**What it does:** Sort DataFrame by one or more columns.

**Key parameters:**
- `by`: Column name(s) to sort by
- `ascending`: True (ascending) or False (descending)
- `na_position`: 'last' (default) or 'first' for NaN values

**When to use:** To order data for analysis, find top/bottom values, or prepare data for visualization.

In [27]:
# Example 1: Sort by score (descending)
sorted_by_score = df.sort_values('score', ascending=False)
print("Top 5 students by score:")
print(sorted_by_score[['name', 'score', 'grade', 'department']].head())

print("\n" + "="*50 + "\n")

# Example 2: Sort by multiple columns
sorted_multi = df.sort_values(['department', 'score'], ascending=[True, False])
print("Sorted by department (ascending) then score (descending):")
print(sorted_multi[['name', 'department', 'score']].head(10))

Top 5 students by score:
     name  score grade department
12    Mia    150     B       Math
16  Quinn     96     A         CS
0   Alice     95     A         CS
8     Ivy     94     A         cs
11   Liam     93     A         CS


Sorted by department (ascending) then score (descending):
           name department  score
16        Quinn         CS     96
0         Alice         CS     95
11         Liam         CS     93
2     Charlie           CS     92
13         Noah         CS     91
5         Frank         CS     90
19         Tina         CS     89
12          Mia       Math    150
4           Eve       Math     88
6         Grace       Math     87


## 11. Removing Duplicates

### 11.1 `df.duplicated()` and `df.drop_duplicates()`

**What it does:**
- `duplicated()`: Returns boolean Series indicating duplicate rows
- `drop_duplicates()`: Removes duplicate rows

**Key parameters:**
- `subset`: Columns to consider when identifying duplicates
- `keep`: 'first' (keep first), 'last' (keep last), or False (drop all duplicates)

**When to use:** To identify and remove duplicate records from your dataset.

In [28]:
# Create a DataFrame with duplicates for demonstration
df_with_duplicates = pd.concat([df, df.iloc[[0, 1, 2]]], ignore_index=True)
print(f"DataFrame with duplicates: {len(df_with_duplicates)} rows")
print(f"Number of duplicates: {df_with_duplicates.duplicated().sum()}")

print("\n" + "="*50 + "\n")

# Drop duplicates
df_no_duplicates = df_with_duplicates.drop_duplicates()
print(f"After dropping duplicates: {len(df_no_duplicates)} rows")

print("\n" + "="*50 + "\n")

# Check for duplicates based on specific columns
duplicates_by_name = df_with_duplicates.duplicated(subset=['name'], keep=False)
print("Rows with duplicate names:")
print(df_with_duplicates[duplicates_by_name][['name', 'student_id']])

DataFrame with duplicates: 23 rows
Number of duplicates: 3


After dropping duplicates: 20 rows


Rows with duplicate names:
           name  student_id
0         Alice           1
1           Bob           2
2     Charlie             3
20        Alice           1
21          Bob           2
22    Charlie             3


## 12. Handling Missing Values

Missing data is common in real-world datasets. Here are methods to handle it:

### 12.1 `df.dropna()` - Remove Rows/Columns with Missing Values

**What it does:** Drops rows or columns containing missing values.

**Key parameters:**
- `axis`: 0 (rows, default) or 1 (columns)
- `how`: 'any' (drop if any missing) or 'all' (drop if all missing)
- `subset`: List of columns to check (only for axis=0)
- `inplace`: If True, modifies DataFrame directly (default: False)

**When to use:** When missing values are not informative and removing them won't bias your analysis.

In [29]:
# Example 1: Drop rows with ANY missing values
df_dropped_any = df.dropna()
print(f"Original rows: {len(df)}")
print(f"After dropping rows with any missing values: {len(df_dropped_any)}")

print("\n" + "="*50 + "\n")

# Example 2: Drop rows with missing values ONLY in 'age' column
df_dropped_age = df.dropna(subset=['age'])
print(f"After dropping rows with missing 'age': {len(df_dropped_age)}")

print("\n" + "="*50 + "\n")

# Example 3: Drop rows where ALL values are missing (rare, but useful)
df_dropped_all = df.dropna(how='all')
print(f"After dropping rows where ALL values are missing: {len(df_dropped_all)}")

Original rows: 20
After dropping rows with any missing values: 17


After dropping rows with missing 'age': 17


After dropping rows where ALL values are missing: 20


### 12.2 Impute Missing Values

You are likely to deal with missing values in you assignments. Generally, there are two strategies: either remove the rows/columns with missing values, or impute the missing values based on what you know about the data. 

Be careful with the imputation, it might introduce bias to your data. Most of the time it is a good idea to keep the original data. If imputation is necessary, add a _new_ column with the imputed values. To start with imputation, you need to understand the missing pattern. Most importantly, you need to identify whether the values are missing at **random** or there is a dependency on other variables. If you use any of the following strategies, you need to justify your choice. 

Here are some common strategies for imputation:

1. **Fill with a constant value**: e.g. `df.fillna(value=...)`
2. **Fill with the mean/median/mode of the column**: e.g. `df.fillna(df.mean())`
3. **Forward fill/Backward fill**: e.g. `df.ffill()` for forward fill, `df.bfill()` for backward fill
4. **Interpolate**: Estimate the missing values based on the neighboring values and a presumed underlying data structure. e.g. `df.interpolate()`





In [30]:
# Example 1: Fill missing values with a constant
df_filled_constant = df.copy()
df_filled_constant['age'] = df_filled_constant['age'].fillna(20)  # Fill with mean age
print("After filling 'age' with 20:")
print(df_filled_constant[['name', 'age']].head(10))

print("\n" + "="*50 + "\n")

# Example 2: Fill with mean (common for numeric columns)
mean_age = df['age'].mean()
df_filled_mean = df.copy()
df_filled_mean['age'] = df_filled_mean['age'].fillna(mean_age)
print(f"After filling 'age' with mean ({mean_age:.2f}):")
print(df_filled_mean[['name', 'age']].head(10))

print("\n" + "="*50 + "\n")

# Example 3: Forward fill (carry last valid value forward)
df_filled_ffill = df.copy()
df_filled_ffill['age'] = df_filled_ffill['age'].ffill() 
print("After forward fill:")
print(df_filled_ffill[['name', 'age']].head(10))

After filling 'age' with 20:
          name   age
0        Alice  20.0
1          Bob  21.0
2    Charlie    20.0
3        Diana  19.0
4          Eve  22.0
5        Frank  20.0
6        Grace  20.0
7        Henry  23.0
8          Ivy  21.0
9         Jack  20.0


After filling 'age' with mean (20.59):
          name        age
0        Alice  20.000000
1          Bob  21.000000
2    Charlie    20.588235
3        Diana  19.000000
4          Eve  22.000000
5        Frank  20.000000
6        Grace  20.588235
7        Henry  23.000000
8          Ivy  21.000000
9         Jack  20.000000


After forward fill:
          name   age
0        Alice  20.0
1          Bob  21.0
2    Charlie    21.0
3        Diana  19.0
4          Eve  22.0
5        Frank  20.0
6        Grace  20.0
7        Henry  23.0
8          Ivy  21.0
9         Jack  20.0


## 13. Handling Outliers

### 13.1 Identifying and Handling Outliers

**What it does:** Detect and handle extreme values that may be errors or need special treatment.

Outliers may bias your analysis. But remember to remove them **only** if you have a good reason to do so. You will need to justify your choice in your report of whether you removed them or not.

**Methods:**
- Statistical methods (IQR, Z-score)
- Domain knowledge
- Visualization


In [31]:
# Example: Identify outliers using IQR method
Q1 = df['score'].quantile(0.25)
Q3 = df['score'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Score statistics:")
print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")

print("\n" + "="*50 + "\n")

# Find outliers
outliers = df[(df['score'] < lower_bound) | (df['score'] > upper_bound)]
print("Outliers detected:")
print(outliers[['name', 'score']])

print("\n" + "="*50 + "\n")

# Option 1: Remove outliers
df_no_outliers = df[(df['score'] >= lower_bound) & (df['score'] <= upper_bound)]
print(f"Original rows: {len(df)}")
print(f"After removing outliers: {len(df_no_outliers)}")

print("\n" + "="*50 + "\n")

# Option 2: Cap outliers (winsorization)
df_capped = df.copy()
df_capped.loc[df_capped['score'] > upper_bound, 'score'] = upper_bound
df_capped.loc[df_capped['score'] < lower_bound, 'score'] = lower_bound
print("After capping outliers:")
print(df_capped[df_capped['name'].isin(outliers['name'])][['name', 'score']])

Score statistics:
Q1: 80.25, Q3: 92.25, IQR: 12.0
Lower bound: 62.25, Upper bound: 110.25


Outliers detected:
   name  score
12  Mia    150


Original rows: 20
After removing outliers: 19


After capping outliers:
   name   score
12  Mia  110.25


/var/folders/pp/s_ymht6j1vl62cxbn9rkd1jh0000gn/T/ipykernel_99236/3695171401.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '110.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_capped.loc[df_capped['score'] > upper_bound, 'score'] = upper_bound


## 14. Combining and Connecting DataFrames

### 14.1 `pd.concat()` - Concatenate DataFrames

**What it does:** Combine DataFrames along an axis (rows or columns).

**Key parameters:**
- `axis`: 0 (concatenate rows) or 1 (concatenate columns)
- `ignore_index`: Reset index after concatenation

**When to use:** To combine datasets with same columns (stacking) or same rows (side-by-side).

In [32]:
# Create two sample DataFrames
df1 = df[['student_id', 'name', 'score']].head(5)
df2 = df[['student_id', 'name', 'score']].tail(5)

print("DataFrame 1:")
print(df1)
print("\nDataFrame 2:")
print(df2)

print("\n" + "="*50 + "\n")

# Concatenate vertically (stack rows)
df_combined = pd.concat([df1, df2], ignore_index=True)
print("After concatenating (stacking rows):")
print(df_combined)

DataFrame 1:
   student_id         name  score
0           1        Alice     95
1           2          Bob     85
2           3    Charlie       92
3           4        Diana     72
4           5          Eve     88

DataFrame 2:
    student_id    name  score
15          16    Paul     84
16          17   Quinn     96
17          18  Rachel     83
18          19     Sam     73
19          20    Tina     89


After concatenating (stacking rows):
   student_id         name  score
0           1        Alice     95
1           2          Bob     85
2           3    Charlie       92
3           4        Diana     72
4           5          Eve     88
5          16         Paul     84
6          17        Quinn     96
7          18       Rachel     83
8          19          Sam     73
9          20         Tina     89


### 13.2 `pd.merge()` - Merge DataFrames

**What it does:** Combine DataFrames based on common columns. It is extremely useful when you have multiple data sources and you need to combine them.

**Key parameters:**
- `on`: Column(s) to join on
- `how`: 'inner', 'left', 'right', or 'outer'
- `left_on` / `right_on`: Different column names in left/right DataFrames

**When to use:** To combine datasets with related information (e.g., student info + grades).

In [33]:
# Create two related DataFrames
student_info = df[['student_id', 'name', 'age', 'department']].head(10)
student_scores = df[['student_id', 'score', 'grade']].head(8)  # Intentionally fewer rows

print("Student Info DataFrame:")
print(student_info)
print("\nStudent Scores DataFrame:")
print(student_scores)

print("\n" + "="*50 + "\n")

# Inner join (only matching records)
inner_merge = pd.merge(student_info, student_scores, on='student_id', how='inner')
print("Inner join (only matching student_ids):")
print(inner_merge)

print("\n" + "="*50 + "\n")

# Left join (keep all from left DataFrame)
left_merge = pd.merge(student_info, student_scores, on='student_id', how='left')
print("Left join (keep all students from info):")
print(left_merge[['student_id', 'name', 'score', 'grade']])

Student Info DataFrame:
   student_id         name   age department
0           1        Alice  20.0         CS
1           2          Bob  21.0       Math
2           3    Charlie     NaN         CS
3           4        Diana  19.0    Physics
4           5          Eve  22.0       Math
5           6        Frank  20.0         CS
6           7        Grace   NaN       Math
7           8        Henry  23.0    Physics
8           9          Ivy  21.0         cs
9          10         Jack  20.0       Math

Student Scores DataFrame:
   student_id  score grade
0           1     95     A
1           2     85     B
2           3     92     A
3           4     72     C
4           5     88     B
5           6     90     a
6           7     87     B
7           8     75     C


Inner join (only matching student_ids):
   student_id         name   age department  score grade
0           1        Alice  20.0         CS     95     A
1           2          Bob  21.0       Math     85     B
2        

## Summary

This tutorial covered essential pandas methods for data cleaning and manipulation:

1. **Data Inspection**: `info()`, `describe()`, `head()`, `tail()`, `isna()`, `value_counts()`
2. **Missing Values**: `dropna()`, `fillna()`
3. **Selection & Filtering**: Column selection, boolean indexing
4. **Transformation**: Creating columns, `apply()`, `map()`, `replace()`
5. **String Operations**: `str.strip()`, `str.upper()`, `str.contains()`
6. **Type Conversion**: `to_datetime()`, `astype()`
7. **Grouping**: `groupby()` with aggregations
8. **Sorting**: `sort_values()`
9. **Duplicates**: `duplicated()`, `drop_duplicates()`
10. **Outliers**: Detection and handling methods
11. **Combining Data**: `concat()`, `merge()`

**Best Practices:**
- Always inspect your data first
- Make copies before modifying (use `.copy()`)
- Document your cleaning decisions and present them in your report.
- Check data types and convert as needed
- Handle missing values thoughtfully
- Validate data quality (outliers, duplicates, formats)

**Next Steps:**
- Practice with your own datasets
- Explore pandas documentation for advanced methods
- Learn about `pd.pivot_table()` for reshaping data
- Study `pd.melt()` and `pd.pivot()` for wide/long format conversions